# Memory Registry Analysis
Another area to investigate in memory is the registry data resident which can provide useful informaiton on host configuration, user activity, and malware persistence mechanism. Where is memory Stored

* C:\Windows\System32\config
* Windows 7

The common registry file are:

* SYSTEM
* SOFTWARE
* SECURITY
* NTUSER
* USERCLASS
* SAM

These files are refered to as "hives". These file are not human readable and need to be parsed with special tools. These files are best acquired forensically or data extracted through memory. The registry keeps track of informaiton in Keys and values that contain data. Keys in the registry are analhous to file paths, values to files and data as file content. The SYSTEM and SOFTWARE registry file contain the bulk of the system configuration settings.

The Registry only has one timestamp which is assocated with the last write time of the Key, so if a Key has multiple values and only one changes the last write is only associated with the key not the value or the data. 

You can see where registry is stored in memory by execution the following volatility plugin:

In [12]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 hivelist

Volatility Foundation Volatility Framework 2.6
Virtual    Physical   Name
---------- ---------- ----
0xe1b01718 0x0cdce718 \Device\HarddiskVolume1\Documents and Settings\demo\Local Settings\Application Data\Microsoft\Windows\UsrClass.dat
0xe1a47b60 0x0c073b60 \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT
0xe190e008 0x0bfa9008 \Device\HarddiskVolume1\Documents and Settings\LocalService\Local Settings\Application Data\Microsoft\Windows\UsrClass.dat
0xe1905b60 0x0bf2cb60 \Device\HarddiskVolume1\Documents and Settings\LocalService\NTUSER.DAT
0xe1830b60 0x0a03fb60 \Device\HarddiskVolume1\Documents and Settings\NetworkService\Local Settings\Application Data\Microsoft\Windows\UsrClass.dat
0xe1836b60 0x0abaeb60 \Device\HarddiskVolume1\Documents and Settings\NetworkService\NTUSER.DAT
0xe14b56b8 0x08a726b8 \Device\HarddiskVolume1\WINDOWS\system32\config\software
0xe1739710 0x085d0710 \Device\HarddiskVolume1\WINDOWS\system32\config\default
0xe173eb60 0x08594b60 \Device\HarddiskVo


One of the first items to collect when performing analysis is to gather basic host configutation information to ensure you are examining the right system and document what system has what findings. Most of the configuration information needing to be documented comes from the registry such as the following:

* Hostname
* Domain
* IP address
* Network assignment protocol

To get this inforamtion from the registry using volatilty you will use the `printkey` command and the specified registry key where the configuration data resides. The registry stores configuration information in multiple locations and can have multiple copies of this configuration as backup if a system crashes or gets corrupted in some manner. The registry documents what configuration information is current by the key `HKLM\CurrentControlSet`. To get this setting execute the following command:

## Current Control Set

In [1]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "CurrentControlSet"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: CurrentControlSet (V)
Last updated: 2009-04-16 16:09:51 UTC+0000

Subkeys:

Values:
REG_LINK      SymbolicLinkValue : (V) \Registry\Machine\System\ControlSet001


You will use this value as input for the rest of the specific `printkey` commands for the active control set. 

## Hostname

In [4]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Control\ComputerName\ComputerName"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: ComputerName (S)
Last updated: 2008-08-11 22:47:29 UTC+0000

Subkeys:

Values:
REG_SZ        ComputerName    : (S) VICTIM1 


## Domain and DHCP Configurations

In [5]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Services\Tcpip\Parameters"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: Parameters (S)
Last updated: 2009-02-17 04:13:18 UTC+0000

Subkeys:
  (S) Adapters
  (S) DNSRegisteredAdapters
  (S) Interfaces
  (S) PersistentRoutes
  (S) Winsock

Values:
REG_SZ        NV Hostname     : (S) victim1 
REG_EXPAND_SZ DataBasePath    : (S) %SystemRoot%\System32\drivers\etc 
REG_SZ        NameServer      : (S)  
REG_DWORD     ForwardBroadcasts : (S) 0
REG_DWORD     IPEnableRouter  : (S) 0
REG_SZ        Domain          : (S)  
REG_SZ        Hostname        : (S) victim1 
REG_SZ        SearchList      : (S)  
REG_DWORD     UseDomainNameDevolution : (S) 1
REG_DWORD     EnableICMPRedirect : (S) 1
REG_DWORD     DeadGWDetectDefault : (S) 1
REG_DWORD     DontAddDefaultGatewayDefault : (S) 0
REG_DWORD     EnableSecurityFilters : (S) 0
REG_DWORD     TcpWindowSize   : (S) 64240


## Network Interfaces

In [6]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Services\Tcpip\Parameters\Interfaces"#

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: Interfaces (S)
Last updated: 2009-02-16 22:17:22 UTC+0000

Subkeys:
  (S) {5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}
  (S) {8E96184A-2A18-4D1C-AA97-4F126156F2EF}
  (S) {942691E0-1A9A-4F39-B9B4-7D077FC9B26B}
  (S) {C2513D1F-B099-4423-9C77-A76C525BF24C}

Values:


## Parse information in each Interface

In [7]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: {5B00ABC6-D4D3-46B2-9C5D-5618273C35F6} (S)
Last updated: 2009-02-17 19:13:50 UTC+0000

Subkeys:

Values:
REG_DWORD     UseZeroBroadcast : (S) 0
REG_DWORD     EnableDeadGWDetect : (S) 1
REG_DWORD     EnableDHCP      : (S) 0
REG_MULTI_SZ  IPAddress       : (S) ['192.168.157.10', '', '']
REG_MULTI_SZ  SubnetMask      : (S) ['255.255.255.0', '', '']
REG_MULTI_SZ  DefaultGateway  : (S) ['192.168.157.2', '', '']
REG_MULTI_SZ  DefaultGatewayMetric : (S) ['0', '', '']
REG_SZ        NameServer      : (S)  
REG_SZ        Domain          : (S)  
REG_DWORD     RegistrationEnabled : (S) 1
REG_DWORD     RegisterAdapterName : (S) 0
REG_MULTI_SZ  TCPAllowedPorts : (S) ['0', '', '']
REG_MULTI_SZ  UDPAllowedPorts : (S) ['0', '', '']
REG_MULTI_SZ  RawIPAllowedProtocols : (S) ['0', '', '']
REG_MULTI_SZ  NTEContextList 

In [8]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: {8E96184A-2A18-4D1C-AA97-4F126156F2EF} (S)
Last updated: 2008-08-11 20:21:19 UTC+0000

Subkeys:

Values:
REG_DWORD     UseZeroBroadcast : (S) 0
REG_DWORD     EnableDHCP      : (S) 0
REG_MULTI_SZ  IPAddress       : (S) ['0.0.0.0', '', '']
REG_MULTI_SZ  SubnetMask      : (S) ['0.0.0.0', '', '']
REG_MULTI_SZ  DefaultGateway  : (S) ['', '']
REG_DWORD     EnableDeadGWDetect : (S) 1
REG_DWORD     DontAddDefaultGateway : (S) 0


In [9]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: {942691E0-1A9A-4F39-B9B4-7D077FC9B26B} (S)
Last updated: 2008-08-11 20:21:19 UTC+0000

Subkeys:

Values:
REG_DWORD     UseZeroBroadcast : (S) 0
REG_DWORD     EnableDHCP      : (S) 0
REG_MULTI_SZ  IPAddress       : (S) ['0.0.0.0', '', '']
REG_MULTI_SZ  SubnetMask      : (S) ['0.0.0.0', '', '']
REG_MULTI_SZ  DefaultGateway  : (S) ['', '']
REG_DWORD     EnableDeadGWDetect : (S) 1
REG_DWORD     DontAddDefaultGateway : (S) 0


In [10]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: {C2513D1F-B099-4423-9C77-A76C525BF24C} (S)
Last updated: 2009-02-05 17:15:56 UTC+0000

Subkeys:

Values:
REG_DWORD     UseZeroBroadcast : (S) 0
REG_DWORD     EnableDeadGWDetect : (S) 1
REG_DWORD     EnableDHCP      : (S) 0
REG_MULTI_SZ  IPAddress       : (S) ['192.168.157.10', '', '']
REG_MULTI_SZ  SubnetMask      : (S) ['255.255.255.0', '', '']
REG_MULTI_SZ  DefaultGateway  : (S) ['192.168.157.2', '', '']
REG_MULTI_SZ  DefaultGatewayMetric : (S) ['0', '', '']
REG_SZ        NameServer      : (S)  
REG_SZ        Domain          : (S)  
REG_DWORD     RegistrationEnabled : (S) 1
REG_DWORD     RegisterAdapterName : (S) 0
REG_MULTI_SZ  TCPAllowedPorts : (S) ['0', '', '']
REG_MULTI_SZ  UDPAllowedPorts : (S) ['0', '', '']
REG_MULTI_SZ  RawIPAllowedProtocols : (S) ['0', '', '']
REG_MULTI_SZ  NTEContextList 

## Timezone

In [30]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "ControlSet001\Control\TimeZoneInformation"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\system
Key name: TimeZoneInformation (S)
Last updated: 2009-03-17 17:07:55 UTC+0000

Subkeys:

Values:
REG_DWORD     Bias            : (S) 300
REG_SZ        StandardName    : (S) Eastern Standard Time 
REG_DWORD     StandardBias    : (S) 0
REG_BINARY    StandardStart   : (S) 
0x00000000  00 00 0b 00 01 00 02 00 00 00 00 00 00 00 00 00   ................
REG_SZ        DaylightName    : (S) Eastern Daylight Time 
REG_DWORD     DaylightBias    : (S) 4294967236
REG_BINARY    DaylightStart   : (S) 
0x00000000  00 00 03 00 02 00 02 00 00 00 00 00 00 00 00 00   ................
REG_DWORD     ActiveTimeBias  : (S) 240


### Basic Host Information
|Hostname     |IP     |Domain     |Timezone|
|-------------|-------|-----------|--------|
|victim1|192.168.157.10|N/A|EST|

## Persistence

The registry houses the largest set of persistence mechansims on Windows. There are tens if not hundreds of registry keys which can be abused my malware to maintain persistence. Common persistence mechanisms in the registry are the following:

* HKLM\Software\Microsoft\Windows\CurrentVersion\Run
* HKLM\Software\Microsoft\Windows\CurrentVersion\RunOnce
* HKLM\Microsoft\Windows\CurrentVersion\RunServices
* HKCU\Software\Microsoft\Windows\CurrentVersion\Run
* HKCU\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\Run

These keys can be enumerated from memory by calling each key to review the values. An example of listing these keys content is executed in the following plugin:

In [24]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 printkey -K "Microsoft\Windows\CurrentVersion\Run"

Volatility Foundation Volatility Framework 2.6
Legend: (S) = Stable   (V) = Volatile

----------------------------
Registry: \Device\HarddiskVolume1\WINDOWS\system32\config\software
Key name: Run (S)
Last updated: 2009-01-27 04:59:59 UTC+0000

Subkeys:

Values:
REG_SZ        porkbun         : (S) C:\WINDOWS\system32\exploder.exe                                                                                                                                                                                                                               
REG_SZ        scardsrv        : (S) C:\WINDOWS\system32\scardsrv.exe 
REG_SZ        VMware Tools    : (S) C:\Program Files\VMware\VMware Tools\VMwareTray.exe 
REG_SZ        VMware User Process : (S) C:\Program Files\VMware\VMware Tools\VMwareUser.exe 


Since there are so many keys to review it can become laborious to review every single key but luckily there are custom developed volatility plugins to help with this.  

### Autoruns
Autoruns is a custom volatility plugin that enumerates most of the Windows registry persistence mechanisms. This plugin will query the possible keys and execute the print plugin for you. This plugin can be found here:

* https://github.com/tomchop/volatility-autoruns

To activate the plugin download the python script and put it into your volatilty source folder in the `plugin` directory. You can use this method to either write your own plugins or download others custom plugins. A link to commonly used and communitiy developed volatility plugins not in the main source can be found here:

* https://github.com/volatilityfoundation/community

To run the `autoruns` plugin execution the following command:

In [13]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 autoruns

Volatility Foundation Volatility Framework 2.6
ERROR   : volatility.debug    : You must specify something to do (try -h)


### Services

In [25]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 svcscan

Volatility Foundation Volatility Framework 2.6
Offset: 0x381e90
Order: 1
Start: SERVICE_DISABLED
Process ID: -
Service Name: Abiosdsk
Display Name: Abiosdsk
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x381f20
Order: 2
Start: SERVICE_DISABLED
Process ID: -
Service Name: abp480n5
Display Name: abp480n5
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x381fb0
Order: 3
Start: SERVICE_BOOT_START
Process ID: -
Service Name: ACPI
Display Name: Microsoft ACPI Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\ACPI

Offset: 0x382038
Order: 4
Start: SERVICE_DISABLED
Process ID: -
Service Name: ACPIEC
Display Name: ACPIEC
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x3820c8
Order: 5
Start: SERVICE_DISABLED
Process ID: -
Service Name: adpu160m
Display Name: adpu160m
Service Type: SERVICE_KERNEL_DRIVER
Service State: SE

### Shimcache

In [18]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 shimcache

Volatility Foundation Volatility Framework 2.6
Last Modified                  Last Update                    Path
------------------------------ ------------------------------ ----
2007-11-30 11:18:51 UTC+0000   2009-02-16 23:11:27 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\e76b316b6389286fbb342d033e63f1ba\update\update.exe
2008-04-14 12:00:00 UTC+0000   2009-02-24 20:41:29 UTC+0000   \??\C:\WINDOWS\system32\verclsid.exe
2007-11-30 11:20:44 UTC+0000   2009-02-16 23:10:48 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\f3174104a45ae9b1276d8609df91dcb9\update\update.exe
2008-04-14 12:00:00 UTC+0000   2009-04-13 20:36:12 UTC+0000   \??\C:\WINDOWS\system32\wscntfy.exe
2008-04-14 12:00:00 UTC+0000   2009-02-24 20:41:28 UTC+0000   \??\C:\WINDOWS\System32\cscui.dll
2008-03-20 19:41:20 UTC+0000   2009-02-16 23:10:59 UTC+0000   \??\c:\d9a0b9ff409f8d556fe7\update\update.exe
2008-09-16 19:40:27 UTC+0000   2009-02-16 23:10:57 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Down

## User Activity
### User Assit

In [15]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 userassist

Volatility Foundation Volatility Framework 2.6
----------------------------
Registry: \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT 
Path: Software\Microsoft\Windows\CurrentVersion\Explorer\UserAssist\{0D6D4F41-2994-4BA0-8FEF-620E43CD2812}\Count
Last updated: 2009-02-16 23:02:31 UTC+0000

Subkeys:

Values:

REG_BINARY    UEME_CTLSESSION : Raw Data:
0x00000000  55 84 4c 0e 02 00 00 00                           U.L.....
----------------------------
Registry: \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT 
Path: Software\Microsoft\Windows\CurrentVersion\Explorer\UserAssist\{5E6AB780-7743-11CF-A12B-00AA004AE837}\Count
Last updated: 2009-05-04 22:09:59 UTC+0000

Subkeys:

Values:

REG_BINARY    UEME_CTLSESSION : Raw Data:
0x00000000  28 68 4e 0e 11 00 00 00                           (hN.....

REG_BINARY    UEME_CTLCUACount:ctor : 
ID:             1
Count:          2
Last updated:   1970-01-01 00:00:00 UTC+0000
Raw Data:
0x00000000  01 00 00 00 02 00 00 00 00

### Shellbags

In [16]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 shellbags

Volatility Foundation Volatility Framework 2.6
Scanning for registries....
Gathering shellbag items and building path tree...
***************************************************************************
Registry: \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT 
Key: Software\Microsoft\Windows\Shell\Bags\1\Desktop
Last updated: 2009-05-05 19:28:35 UTC+0000
Value                     File Name      Modified Date                  Create Date                    Access Date                    File Attr                 Unicode Name
------------------------- -------------- ------------------------------ ------------------------------ ------------------------------ ------------------------- ------------
ItemPos800x600(1)         AGENT-~1.500   2009-04-16 16:10:56 UTC+0000   2009-04-16 14:03:00 UTC+0000   2009-04-16 16:10:56 UTC+0000   DIR                       Agent-1.3.500-SIGNED 
ItemPos800x600(1)         AGENT-~1.900   2009-05-04 22:09:40 UTC+0000   2009-05-04 22:08:24 UTC+0000